# Multi Agent System to get the most popular news in Singapore

              +----------------+
              | Manager agent  |
              +----------------+
                       |
        _______________|______________
       |                              |
Code Interpreter            +------------------+
    tool                    | Web Search agent |
                            +------------------+
                               |            |
                        Web Search tool     |
                                   Visit webpage tool

In [ ]:
%pip install markdownify

: 

In [ ]:
import os
from dotenv import load_dotenv


: 

In [ ]:
result = load_dotenv()
print(result)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
print(OPENAI_API_KEY)

### Create the web search tool

We need to be able to peek into page found by the native WebSearchTool (Google search equivalent). Below is the tool we use to do so

In [ ]:
import re
import requests
from markdownify import markdownify
from requests.exceptions import RequestException
from smolagents import tool


@tool
def visit_webpage(url: str) -> str:
    """Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Convert the HTML content to Markdown
        markdown_content = markdownify(response.text).strip()

        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

Test our visit webpage tool

In [ ]:
print(visit_webpage("https://ffxiv.consolegameswiki.com/wiki/Dark_Knight")[:500])

### Build our multi-agent system

- Use ToolCalling Agent because web browsing is a single timeline task, so we don't need parallel calls

In [ ]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    InferenceClientModel,
    WebSearchTool,
    LiteLLMModel,
    OpenAIServerModel
)

model_id = "gpt-4.1-mini"

model = OpenAIServerModel(model_id=model_id, api_key=OPENAI_API_KEY)

web_agent = ToolCallingAgent(
    tools=[WebSearchTool(), visit_webpage],
    model=model,
    max_steps=10,
    name="web_search_agent",
    description="Runs web searches for you.",
)

### Create the manager agent to call the web agent we made above

Since this agent is the one tasked with the planning and thinking, advanced reasoning will be beneficial, so a CodeAgent will work well.


In [ ]:
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[web_agent],
    additional_authorized_imports=["time", "numpy", "pandas"],
)

We give some python packages to the Code Agent so it helps it do the stuff we want it to do

In [ ]:
answer = manager_agent.run("Get the most viewed Singapore news today from Straits Times and Reddit. Format it into an array of json object with the its title and summary", max_steps=5)

### Result

We see the conclusion at the Final Answer portion of the output!

# Reference

https://huggingface.co/docs/smolagents/en/examples/multiagents